# Multiplier TEST code

fixed number to stochastic conversion & multiply & stochastic to fixed number conversion

## Import

In [1]:
import random, struct, math
import torch
import numpy as np
import torch.nn as nn 
import argparse
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Parser

In [2]:
parser = argparse.ArgumentParser(description='fixed_mac')
parser.add_argument('--full_bits', type=int, default=8, help='Number of Quantization Bits')
parser.add_argument('--frac_bits', type=int, default=4, help='Number of Quantization Bits')
parser.add_argument('--bBW', type=int, default=4, help='Number of bit width')
args = parser.parse_args(args=[])

## Definition

### int2bin : int number to binary number

In [3]:
def int2bin(iIn,iBW):
    iBW = iBW + 1
    if iIn >= 0:
        bOut = bin(iIn).replace('0b','').rjust(iBW,'0')
    else :
        bOut = bin(iIn & (pow(2,iBW)-1)).replace('0b','').rjust(iBW,'1')
    return bOut[1:]

### XOR : xor gate with string

In [4]:
def XOR(iA,iB):
    if iA != iB :
        iOut = '1'
    else : 
        iOut = '0'
    return iOut

### SNUM : sign number determination

In [5]:
def snum(a):
    if a >= 0 :
        return '0'
    else :
        return '1'

### fxp & flp2fix : floating number to fixed number conversion

In [6]:
class fxp:
    def __init__(self, bIn, iBWF):
        self.iFullBW = len(bIn)
        self.iIntgBW = self.iFullBW - iBWF
        self.bSign = bIn[0]
        self.bIntg = bIn[:self.iIntgBW]
        self.bFrac = bIn[self.iIntgBW:]
        self.fFull = 0
        try:
            for idx, bit in enumerate(bIn):
                if idx == 0:
                    self.fFull = self.fFull + int(bit,2) * -pow(2, self.iIntgBW - 1)
                else:
                    self.fFull = self.fFull + int(bit,2) * pow(2, self.iIntgBW - 1 - idx)
        except:
            print(bIn)
        self.dispFull = self.bIntg +"."+ self.bFrac 
        return

In [7]:
class flp2fix:
    def __init__(self, fIn, iBW, iBWF):
        self.fMin = - 2 ** (iBW - iBWF - 1)
        self.fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
        self.fResol = 2 ** -iBWF
        if fIn < self.fMin or fIn > self.fMax:
            print(f'({fIn}): Out of input range ({self.fMax}/{self.fMin}) during flp -> fix converting ')
        self.iBW = iBW
        self.iBWI = iBW - iBWF
        self.iBWF = iBWF

        self.iFLP2INT = abs(int(fIn * 2 ** iBWF))
        if fIn < 0:
            self.iFLP2INT = 2 ** (iBW-1) - self.iFLP2INT

        if fIn >= 0:
            self.bFull = bin(self.iFLP2INT)[2:].rjust(iBW, '0')
        else:
            self.bFull = '1'+bin(self.iFLP2INT)[2:].rjust(iBW-1, '0')
            if len(self.bFull) > iBW:
                self.bFull = '0' * iBW

        self.cssFxp = fxp(self.bFull, self.iBWF)
        self.bSign = self.cssFxp.bSign
        self.bIntg = self.cssFxp.bIntg
        self.bFrac = self.cssFxp.bFrac
        self.fFull = self.cssFxp.fFull
        return

### LFSR : make pseudorandom number bitstream

In [8]:
class LFSR:
    def Random(self):
        self.b0 = eval(f'str(random.randint(0,1))')
        self.b1 = eval(f'str(random.randint(0,1))')
        self.b2 = eval(f'str(random.randint(0,1))')
        self.b3 = eval(f'str(random.randint(0,1))')
        
        return self.b0 + self.b1 + self.b2 + self.b3
    
    def Normal(self,stream):
        self.b0 = XOR(int(stream[2]),int(stream[3]))
        self.b1 = stream[0]
        self.b2 = stream[1]
        self.b3 = stream[2]
        
        return self.b0 + self.b1 + self.b2 + self.b3
    
    def Allzero(self):
        self.b0 = '0'
        self.b1 = '0'
        self.b2 = '0'
        self.b3 = '0'
        
        return self.b0 + self.b1 + self.b2 + self.b3

### LFSRlist : make pseudorandom number bitstream with 2**bBW cycle

In [9]:
def LFSRlist():
    lfsr = LFSR()
    lfsrlist = []
    for k in range(2**(args.bBW)-1): #lfsr number generating
        if k == 0:
            lfsrlist.append(lfsr.Random())
        else :
            lfsrlist.append(lfsr.Normal(lfsrlist[k-1]))
        if (k == 2**(args.bBW)-2):
            lfsrlist.append(lfsr.Allzero())
    
    if (args.bBW) != args.frac_bits :
        if args.bBW < args.frac_bits :
            for i in range(len(lfsrlist)):
                lfsrlist[i] = lfsrlist[i] + (args.frac_bits-args.bBW)*'0'
        else :
            print("it can't work")
            return 0
    
    return lfsrlist

### Comp : Comparator in SNG

In [10]:
def Comp(a,lfsr,snum):
    for com in range(0,len(a)):
        oA = '0'
        if a[com]!=lfsr[com]:
            if(int(a[com]) > int(lfsr[com])):
                oA = '1'
            break
    return XOR(oA,snum)

### Perm : module for permutation SNG 

In [11]:
def perm(a):
    al = len(a)
    blist = []
    for i in range(al) :
        #print(al-i-1)
        blist.append(a[al-i-1])
    
    b = "".join(blist)
    
    return b

### findMaxMin : find abs max value in act tensor

In [12]:
def findMaxMin(data):
    max = torch.max(data)
    min = torch.min(data)
    SF=torch.max(abs(max),abs(min)).item()
    return SF

### SNG : stochastic number generator module

In [13]:
def SNG(iIN,lfsr):

    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        lNUM = lfsr[k]
        a = Comp(bIN[-(args.frac_bits):],lNUM,sNUM)
        oAlist.append(a) #comparator of input a
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    if sNUM == '1' and bIN == args.full_bits*'0' :
        return '10000000000000000'
    else :
        return sA

### SNG_P : permutation stochastic number generator module

In [14]:
def SNG_P(iIN,lfsr):
    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        if (args.bBW == args.frac_bits) :    
            lNUM = perm(lfsr[k])
        elif (args.bBW < args.frac_bits) :
            lNUM = perm(lfsr[k][:args.bBW])+(args.frac_bits-args.bBW)*"0"
        a = Comp(bIN[-(args.frac_bits):],lNUM,sNUM)
        oAlist.append(Comp(bIN[-(args.frac_bits):],lNUM,sNUM)) #comparator of input a
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    if sNUM == '1' and bIN == args.full_bits*'0' :
        return '10000000000000000'
    else :
        return sA

### SNGnumpy : SNG module with numpy

In [15]:
def SNGnumpy(fIn,lfsr):
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(SNG(float(aNumpy),lfsr))
                     
    return np.array(sList).reshape(fIn.size())

### SNGpnumpy : permutation SNG module with numpy

In [16]:
def SNGpnumpy(fIn,lfsr):
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(SNG_P(float(aNumpy),lfsr))
                     
    return  np.array(sList).reshape(fIn.size())

### CountOne : count 1 in stochastic number bit stream

In [17]:
def CountOne(nIn):
    nlist = []
    for num in nIn.reshape(-1):
        n = 0
        for a in num:
            if a == '1' :
                n += 1
        if num[0] == '1' :
            nlist.append(n-1)
        else :
            nlist.append(n)
    return torch.tensor(nlist).view(nIn.shape)

### defSign : sign determination in S2N 

In [18]:
def defSign(nIn):
    nlist = []
    for num in nIn.reshape(-1):
        if num[0] == '1' :
            nlist.append(-1)
        else :
            nlist.append(1)
    return torch.tensor(nlist).view(nIn.shape)

### Multiplier

In [19]:
def mul(a,b):
    al = len(a)
    bl = len(b)
    
    outlist = []
    
    if al != bl :
        print("length of string is different")
        return 0
    
    outlist.append(XOR(a[0],b[0]))
    
    for i in range(al-1) :
        outlist.append(str(int(a[i+1]) & int(b[i+1])))
    
    #print(outlist)
    out = "".join(outlist)
    
    return out    

In [20]:
def defSign1(nIn):
    if nIn[0] == '1' :
        return -1
    else :
        return 1

In [21]:
def CountOne1(nIn):
    n = 0
    for num in nIn:
        if num == '1' :
            n += 1
    if nIn[0] == '1' :
        return n-1
    else :
        return n

In [22]:
def S2None(sIn,SF):
    s = defSign(sIn)
    o = (CountOne(sIn)/(2**args.bBW))*SF*s
    return o

In [23]:
def mulNumpy(aIn,bIn,aSF,wSF):
    mList = []
    for i in range(aIn.shape[0]):
        for j in range(bIn.T.shape[1]):
            sum = 0
            for k in range(aIn.shape[1]):
                sum += S2None(mul((aIn[i][k].astype(str)),(bIn.T)[k][j].astype(str)),aSF*wSF)
            mList.append(sum)
    return torch.tensor(mList).view(aIn.shape[0],bIn.T.shape[1])

In [24]:
def FindError(a,b):
    error = 0
    for anum,bnum in zip(a.view(-1),b.view(-1)):
        e = (abs(bnum-anum)*100).item()
        error += e
    return error/(len(a.view(-1))+len(b.view(-1)))

# TEST : LFSR 4 bit

In [25]:
act = torch.rand((10,15))-torch.randint(4,(10,15))+torch.randint(4,(10,15))
wei = torch.rand((10,15))-torch.randint(4,(10,15))+torch.randint(4,(10,15))

### number to stochastic conversion

In [26]:
aSF = findMaxMin(act)
wSF = findMaxMin(wei)

In [27]:
def N2S(act,wei,aSF,wSF):
    
    lfsrlist = LFSRlist()
    
    a = SNGnumpy(act/aSF,lfsrlist)
    w = SNGpnumpy(wei/wSF,lfsrlist)
    
    return a,w

In [28]:
def N2Sone(data,SF):
    lfsrlist = LFSRlist()
    
    o = SNGnumpy(data/SF,lfsrlist)
    
    return o

### stochastic to number conversion

In [29]:
def S2N(a,w,aSF,wSF):
    
    sa = defSign(a)
    sw = defSign(w)

    ca = (CountOne(a)/(2**args.bBW)) * aSF *sa
    cw = (CountOne(w)/(2**args.bBW)) * wSF *sw

    return ca,cw

In [30]:
a,w = N2S(act,wei,aSF,wSF)

In [31]:
ca,cw = S2N(a,w,aSF,wSF)

In [32]:
def multest(aIn,wIn,aSF,wSF):
    mlist=[]
    out = 0
    for a,b in zip(aIn.reshape(-1),wIn.reshape(-1)) :
        out = mul(a,b)
        mlist.append(out)
    
    return np.array(mlist).reshape(aIn.shape)
    

In [33]:
c = multest(a,w,aSF,wSF)

In [34]:
cc = S2None(c,aSF*wSF)

In [35]:
print(aSF,wSF)

3.969839572906494 3.886542558670044


In [36]:
for i in range(10):
    for j in range(15):
        print(f'act value is            : {act[i][j]:^25}(bin : {flp2fix(act[i][j],args.full_bits,args.frac_bits).bFull:^8}) --> {a[i][j]}')
        print(f'wei value is            : {wei[i][j]:^25}(bin : {flp2fix(wei[i][j],args.full_bits,args.frac_bits).bFull:^8}) --> {w[i][j]}')
        print(f'stochastic mul value is : {cc[i][j]:^25}(bin : {flp2fix(cc[i][j],args.full_bits,args.frac_bits).bFull:^8}) --> {c[i][j]}')
        print(f'fixed mul value is      : {act[i][j]*wei[i][j]:^25}(bin : {flp2fix(act[i][j]*wei[i][j],args.full_bits,args.frac_bits).bFull:^8}) --> {N2Sone(act[i][j]*wei[i][j],aSF*wSF)}')
        print(f'------------------------------------------------------------------------------------------')

act value is            :    -0.5090447664260864   (bin : 11111000) --> 10000000000000000
wei value is            :    1.1893527507781982    (bin : 00010011) --> 01111111111111111
stochastic mul value is :           -0.0           (bin : 00000000) --> 10000000000000000
fixed mul value is      :    -0.6054338216781616   (bin : 11110111) --> 10000000000000000
------------------------------------------------------------------------------------------
act value is            :     1.759420394897461    (bin : 00011100) --> 01111111111111111
wei value is            :    -2.911987066268921    (bin : 11010010) --> 10000000000000000
stochastic mul value is :           -0.0           (bin : 00000000) --> 10000000000000000
fixed mul value is      :    -5.123409271240234    (bin : 10101111) --> 10000001010011100
------------------------------------------------------------------------------------------
act value is            :    -1.2665538787841797   (bin : 11101100) --> 10000000000000000
wei valu